In [1]:
!pip3 install camelot-py[cv] tabula-py

     |████████████████████████████████| 40 kB 16 kB/s             
     |████████████████████████████████| 12.0 MB 1.3 MB/s            
     |████████████████████████████████| 77 kB 3.7 MB/s             
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 242 kB 40.6 MB/s            
     |████████████████████████████████| 11.7 MB 53.6 MB/s            
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=cef1e96afbbb6047231d086896c8bfb016882347bbce55e78e7ea93532a5d573
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [2]:
!pip install pikepdf

     |████████████████████████████████| 2.3 MB 689 kB/s            


In [3]:
!pip install xlwt

     |████████████████████████████████| 99 kB 855 kB/s            


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import xlwt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import camelot 
import tabula
import pikepdf
#pdf = pikepdf.open(file,password='ABDPM5030G')
#pdf.save('book_without_pass.pdf')
import warnings
warnings.filterwarnings('ignore')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
/kaggle/input/contract-notes/2022-03-08-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-09-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-07-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-17-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-04-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-16-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-15-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-18-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-11-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-10-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-14-contract-notes_DM3955.pdf


### Starting the PDF table Extraction, and the helper function used for the same

In [5]:
def clean_df(my_table):
    my_table = my_table[2:]
    my_table.drop([0,1,3,8,12],axis=1,inplace=True)
    return my_table

In [6]:
def select_tables(ca_table):
    calc_tab = pd.DataFrame()
    for table in ca_table:
        if table.df.shape[1] == 13:
            calc_tab = pd.concat([calc_tab,table.df])
    calc_tab = clean_df(calc_tab)
    return calc_tab

In [7]:
def correct_txn(dataframe):
    for index,trans in enumerate(dataframe.transaction):
        if '\n' in trans:
            txn_len = len(trans.split('\n'))
            #print(index)
            for i in range(txn_len):
                dataframe.loc[index+i,'transaction'] = trans.split('\n')[i]
    return dataframe

In [8]:
def correct_qty(dataframe):
    for index,qty in enumerate(dataframe.qty):
        if '\n' in qty:
            txn_len = len(qty.split('\n'))
            #print(index)
            for i in range(txn_len):
                dataframe.loc[index+i,'qty'] = qty.split('\n')[i]
    return dataframe

In [9]:
def un_pass(file):
    directory = '../input/contract-notes/'
    full_path = os.path.join(directory,file)
    pdf = pikepdf.open(full_path,password='ABDPM5030G')
    pdf.save(file)

In [10]:
#The below function takes the raw file and outputs the cleaned dataframe
def get_data(file):
    pdf = pikepdf.open(file,password='ABDPM5030G')
    pdf.save('book_without_pass.pdf')
    ca_tables = camelot.read_pdf('./book_without_pass.pdf',pages='all')
    my_data = select_tables(ca_tables)
    my_data.columns = ['order_no','script','transaction',
                   'qty','price','price_d','price_close','PnL']
    my_data = my_data[my_data.order_no != '']
    my_data = my_data.reset_index()
    my_data.drop('index',axis=1,inplace=True)
    my_data = correct_qty(my_data)
    my_data = correct_txn(my_data)
    my_data.qty = my_data.qty.astype('int')
    my_data.price = my_data.price.astype('float')
    return my_data # Try to take the raw data as much as possible, to avoid losing control

In [11]:
def todays_forward(carry_forward, day_df):
    trade_count = day_df.groupby(['script','transaction'])['qty'].sum().reset_index()
    #Cancelling out the trades of the day
    traded_scripts = trade_count.script.unique()
    transaction = []
    txn_qty = []
    for script in traded_scripts:
        temp = trade_count[trade_count.script == script]
        #which is more?
        if (temp.qty.values[0] > temp.qty.values[-1]):
            x = temp.qty.values[0] - temp.qty.values[-1]
            transaction.append('B')
            txn_qty.append(x)
        else:    
            x = temp.qty.values[-1] - temp.qty.values[0]
            transaction.append('S')
            txn_qty.append(x)
    #Creating the pending trades of the day
    traded_df = pd.DataFrame({'script' : traded_scripts,
                         'txn_type' : transaction,
                         'txn_qty' : txn_qty
                         })
    final_df = traded_df.merge(right=carry_f,left_on='script',right_on='script',how='left')
    final_df['remaining_qty'] = abs(final_df.txn_qty - final_df.qty)
    final_df.drop(['order_no','price','price_d','price_close','PnL'],axis=1,inplace=True)
    return final_df

In [12]:
#decide the script
def locate_script(day_df,script,qty,transaction):
    locating = day_df[(day_df.script == script)&(day_df.transaction == transaction)]
    return locating.iloc[-qty:,:]
            #day_dy[-3:,:] will only work

### Now we know which scripts are finally pending at the day's end

Locate the final trades in that script

In [13]:
#Next is to introduce the scripts that are carried forward for the day. 
#This function to makes the dataframe for the carried forward to this day.

def carried_forward(order_no,script,transaction,qty,price):
    loc_qty = 1
    values=pd.DataFrame({'order_no':order_no,'script':script,
                         'transaction':transaction,'qty':loc_qty,
                         'price':price},index=[0])
    values = values.reindex(list(range(qty)),method='ffill')
    return values

In [14]:
def insert_row(idx, df, df_insert):
    dfA = df.iloc[:idx, ]
    dfB = df.iloc[idx:, ]

    df = dfA.append(df_insert).append(dfB).reset_index(drop = True)

    return df

In [15]:
def make_single_1(dataf,qty,index):
    values = dataf.loc[index]
    values[3] = 1 #Here i am changing the qty to 1
    values=pd.DataFrame([values.values],columns=values.index)
    values = values.reindex(list(range(index,index+qty)),method='ffill')
    return values

In [16]:
#This function finally provides the corrected dataframe for each script, transaction wise
def get_trades(df,script):
    df = df[df.order_no != '0'] #Don't take the forwarded trades
    temp = df[df.script == script]
    buy = temp[temp.transaction == 'B'].reset_index()
    buy.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
    buy.order_no = buy.order_no.astype('int') 
    sell = temp[temp.transaction == 'S'].reset_index()
    sell.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
    sell.order_no = sell.order_no.astype('int') 
    mult_buy = buy[buy.qty > 1]
    mult_sell = sell[sell.qty > 1]
    buy.drop(mult_buy.index,inplace=True)
    sell.drop(mult_sell.index,inplace=True)
    
    #calculate sell items
    for i,qty in mult_sell['qty'].iteritems():
        temp = make_single_1(dataf=mult_sell,qty=qty,index=i)
        update_buy = insert_row(i,sell,temp)
        sell = update_buy
        #print('exit loop')

    #calculate buy items
    for i,qty in mult_buy['qty'].iteritems():
        temp = make_single_1(dataf=mult_buy,qty=qty,index=i)
        update_buy = insert_row(i,buy,temp)
        buy = update_buy
        #print('exit loop')
    
    buy.sort_values('order_no',inplace=True)
    sell.sort_values('order_no',inplace=True)
    
    return buy,sell

In [17]:
def get_script_frame(dataf,script,fwd_txn,fwd_qty,fwd_price):
    buy_scr,sell_scr = get_trades(df=dataf,script=script)
    print(buy_scr.shape,sell_scr.shape)
    scr_forward = carried_forward(order_no='forward',script=script,transaction=fwd_txn,
                                  qty=fwd_qty,price=fwd_price)
    print(scr_forward.shape)
    #scr_forward_close = carried_forward(order_no='close',script=script,transaction=cls_txn,
                                  #qty=cls_qty,price=cls_price)
    if fwd_txn == 'B':
        buy_scr = insert_row(0,buy_scr,scr_forward)
        #sell_scr = insert_row(0,sell_scr,scr_forward_close)
    else:
        sell_scr = insert_row(0,sell_scr,scr_forward)
        #buy_scr = insert_row(0,buy_scr,scr_forward_close)
    print('final trade script buy and sell',buy_scr.shape,sell_scr.shape)
    if (buy_scr.shape[0] != 0) and (sell_scr.shape[0] != 0): #Check if the dataframe have data for merge
        if fwd_txn == 'B': #The buy script forward has to be on the top
            final_comp_scr = buy_scr.merge(right=sell_scr,right_index=True,left_index=True,how='left')
            final_comp_scr['PnL'] = final_comp_scr.price_y - final_comp_scr.price_x
        else:#The sell script forward has to be on the top
            final_comp_scr = sell_scr.merge(right=buy_scr,right_index=True,left_index=True,how='left')
            final_comp_scr['PnL'] = final_comp_scr.price_x - final_comp_scr.price_y
        print('entered')
        return final_comp_scr
        
    elif buy_scr.shape[0] != 0:
        final_comp_scr = buy_scr
        print('entered elif')
        return final_comp_scr
    else:
        final_comp_scr = sell_scr
        print('entered else')
        return final_comp_scr
    

In [18]:
import os
directory = '../input/contract-notes'
scripts = ['SILVERMIC22JUNFUT','SILVERMIC22APRFUT']
APR_FWD = [0,69257,70069,71418,69692,70500,70412,68993,68466,67517,68625]
JUN_FWD = [0,69918,70758,72211,70443,71246,71170,69805,69256,68327,69410]

In [19]:
APR_txn = ['S','B','B','B','S','S','S','S','S','S','S']
JUN_txn = ['S','S','S','S','B','B','B','B','B','B','B']
APR_qty = [0,2,4,3,5,6,6,6,6,6,6]
JUN_qty = [0,2,4,3,6,6,6,6,6,6,6]
dates = []
files = []

In [20]:
for filename in os.listdir(directory):
    files.append(filename)
    temp = filename.split('-')
    new_date = temp[0]+'-'+temp[1]+'-'+temp[2]
    dates.append(new_date)

In [21]:
contract_df = pd.DataFrame({'filename':files,'dates':dates})
contract_df.dates = pd.to_datetime(contract_df.dates)
contract_df.sort_values('dates',inplace=True)
contract_df['Jun_close'] = JUN_FWD
contract_df['Apr_close'] = APR_FWD
contract_df['Apr_txn'] = APR_txn
contract_df['Jun_txn'] = JUN_txn
contract_df['Apr_qty'] = APR_qty
contract_df['Jun_qty'] = JUN_qty

In [22]:
contract_df.reset_index(inplace=True)
contract_df.drop('index',inplace=True,axis=1)

In [23]:
#Checking the built dataframe
contract_df

,filename,dates,Jun_close,Apr_close,Apr_txn,Jun_txn,Apr_qty,Jun_qty
0,2022-03-04-contract-notes_DM3955.pdf,2022-03-04,0,0,S,S,0,0
1,2022-03-07-contract-notes_DM3955.pdf,2022-03-07,69918,69257,B,S,2,2
2,2022-03-08-contract-notes_DM3955.pdf,2022-03-08,70758,70069,B,S,4,4
3,2022-03-09-contract-notes_DM3955.pdf,2022-03-09,72211,71418,B,S,3,3
4,2022-03-10-contract-notes_DM3955.pdf,2022-03-10,70443,69692,S,B,5,6
5,2022-03-11-contract-notes_DM3955.pdf,2022-03-11,71246,70500,S,B,6,6
6,2022-03-14-contract-notes_DM3955.pdf,2022-03-14,71170,70412,S,B,6,6
7,2022-03-15-contract-notes_DM3955.pdf,2022-03-15,69805,68993,S,B,6,6
8,2022-03-16-contract-notes_DM3955.pdf,2022-03-16,69256,68466,S,B,6,6
9,2022-03-17-contract-notes_DM3955.pdf,2022-03-17,68327,67517,S,B,6,6


In [24]:
for files in contract_df.filename:
    print(files)
    un_pass(files)

2022-03-04-contract-notes_DM3955.pdf
2022-03-07-contract-notes_DM3955.pdf
2022-03-08-contract-notes_DM3955.pdf
2022-03-09-contract-notes_DM3955.pdf
2022-03-10-contract-notes_DM3955.pdf
2022-03-11-contract-notes_DM3955.pdf
2022-03-14-contract-notes_DM3955.pdf
2022-03-15-contract-notes_DM3955.pdf
2022-03-16-contract-notes_DM3955.pdf
2022-03-17-contract-notes_DM3955.pdf
2022-03-18-contract-notes_DM3955.pdf


In [25]:
directory = '../input/contract-notes/'
for index,row in contract_df.iterrows():
    print(index)
    full_path = os.path.join(directory,row['filename'])
    print('full_path is {} --'.format(full_path))
    contract = get_data(full_path) #get the contract converted to data frame
    print('complete contract data shape is {}'.format(contract.shape))
    collect_jun = get_script_frame(dataf=contract,script='SILVERMIC22JUNFUT',
                              fwd_txn=row['Jun_txn'],fwd_qty=row['Jun_qty'],fwd_price=row['Jun_close'])
    print('collect jun data shape is {}'.format(collect_jun.shape))
    collect_apr = get_script_frame(dataf=contract,script='SILVERMIC22APRFUT',
                              fwd_txn=row['Apr_txn'],fwd_qty=row['Apr_qty'],fwd_price=row['Apr_close'])
    print('collect apr data shape is {}'.format(collect_apr.shape))
    #build the excel file name
    excel = row['filename'].split('-')
    xl_file_name_jun = 'file_name_jun'+excel[2]+'.xls'
    xl_file_name_apr = 'file_name_apr'+excel[2]+'.xls'
    
    #create excel files from collected data frames
    collect_apr.to_excel(xl_file_name_apr)
    collect_jun.to_excel(xl_file_name_jun)
    print('Written the excel file for the {} dataset'.format(row['filename']))

0
full_path is ../input/contract-notes/2022-03-04-contract-notes_DM3955.pdf --


2022-03-19T05:32:09 - INFO - Processing page-1
2022-03-19T05:32:11 - INFO - Processing page-2


complete contract data shape is (18, 8)
(0, 5) (2, 5)
(0, 5)
final trade script buy and sell (0, 5) (2, 5)
entered else
collect jun data shape is (2, 5)
(6, 5) (4, 5)
(0, 5)
final trade script buy and sell (6, 5) (4, 5)
entered
collect apr data shape is (4, 11)
Written the excel file for the 2022-03-04-contract-notes_DM3955.pdf dataset
1
full_path is ../input/contract-notes/2022-03-07-contract-notes_DM3955.pdf --


2022-03-19T05:32:16 - INFO - Processing page-1
2022-03-19T05:32:20 - INFO - Processing page-2
2022-03-19T05:32:24 - INFO - Processing page-3
2022-03-19T05:32:25 - INFO - Processing page-4


complete contract data shape is (58, 8)
(0, 5) (2, 5)
(2, 5)
final trade script buy and sell (0, 5) (4, 5)
entered else
collect jun data shape is (4, 5)
(22, 5) (20, 5)
(2, 5)
final trade script buy and sell (24, 5) (20, 5)
entered
collect apr data shape is (24, 11)
Written the excel file for the 2022-03-07-contract-notes_DM3955.pdf dataset
2
full_path is ../input/contract-notes/2022-03-08-contract-notes_DM3955.pdf --


...

complete contract data shape is (46, 8)
(12, 5) (12, 5)
(6, 5)
final trade script buy and sell (18, 5) (12, 5)
entered
collect jun data shape is (18, 11)
(10, 5) (10, 5)
(6, 5)
final trade script buy and sell (10, 5) (16, 5)
entered
collect apr data shape is (16, 11)
Written the excel file for the 2022-03-16-contract-notes_DM3955.pdf dataset
9
full_path is ../input/contract-notes/2022-03-17-contract-notes_DM3955.pdf --


2022-03-19T05:34:02 - INFO - Processing page-1
2022-03-19T05:34:06 - INFO - Processing page-2
2022-03-19T05:34:09 - INFO - Processing page-3


complete contract data shape is (46, 8)
(19, 5) (19, 5)
(6, 5)
final trade script buy and sell (25, 5) (19, 5)
entered
collect jun data shape is (25, 11)
(5, 5) (5, 5)
(6, 5)
final trade script buy and sell (5, 5) (11, 5)
entered
collect apr data shape is (11, 11)
Written the excel file for the 2022-03-17-contract-notes_DM3955.pdf dataset
10
full_path is ../input/contract-notes/2022-03-18-contract-notes_DM3955.pdf --


2022-03-19T05:34:12 - INFO - Processing page-1
2022-03-19T05:34:13 - INFO - Processing page-2


complete contract data shape is (12, 8)
(1, 5) (2, 5)
(6, 5)
final trade script buy and sell (7, 5) (2, 5)
entered
collect jun data shape is (7, 11)
(5, 5) (4, 5)
(6, 5)
final trade script buy and sell (5, 5) (10, 5)
entered
collect apr data shape is (10, 11)
Written the excel file for the 2022-03-18-contract-notes_DM3955.pdf dataset


for index, row in contract_df.iterrows():
    #getting the raw file into the list
    notes_data.append(get_data(os.path.join(directory,row['filename'])))
    print('completed collecting {} data'.format(row['filename']))